In [52]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

import sys
import json
sys.path.append("..")
from reghub_pack.general_functions import *

In [53]:
# Import data
# Access aws credentials from json file
with open("../aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Specify s3 bucket
bucket = "fs-reghub-news-analysis"

# Connect to aws and dowload the files
aws = awsOps(aws_creds_json)
df = aws.get_df(bucket=bucket, file="data_rule_labels_v1.csv")
df_categories = aws.get_df(bucket=bucket, file="rule_labels_v1.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df[['news_content','rule_labels_comb']]
df=df[~df['rule_labels_comb'].isna()]
df=df[(df['rule_labels_comb'].apply(len)!=2)]
df = df.reset_index()
del df['index']
df

,news_content,rule_labels_comb
0,Berenberg Bank analysts have provided a buy ra...,['market']
1,"The article states that Berenberg, a German in...",['market']
2,"In their analysis on October 30, 2023, experts...",['market']
3,The private bank Berenberg has upgraded its ra...,"['papers', 'market']"
4,In a research note published by Sebastian Bray...,"['sanctions', 'papers', 'market']"
...,...,...
5517,INTERVIEW Interview with Les Échos Interview w...,"['legal', 'statements', 'guidelines']"
5518,UBS's latest Investor Watch report reveals tha...,"['reports', 'personnel']"
5519,SNB erwartet für 2021 Jahresgewinn von rund 26...,"['legal', 'reports', 'market']"
5520,0:00 News A cryptocurrency exchange in Hong Ko...,"['legal', 'reports', 'guidelines', 'press']"


In [54]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [55]:
tokenizer.padding_side = "right" 
tokenizer.pad_token = tokenizer.eos_token

In [56]:
class Dataset(Dataset):
    def __init__(self,lm_dataset):
        self.lm_dataset=lm_dataset
        self.input_ids=[tokenizer(x,max_length=128,padding='max_length',truncation=True,return_tensors="pt") for x in self.lm_dataset['news_content']]
        self.labels=[tokenizer(x,max_length=128,padding='max_length',truncation=True,return_tensors="pt") for x in self.lm_dataset['news_content']]
        
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Tokenize and encode the sequence
        return self.input_ids[idx], self.labels[idx]

In [57]:
dataset=Dataset(df)
dataset[4]

({'input_ids': tensor([[  818,   257,  2267,  3465,  3199,   416, 26190, 43050,   347, 14226,
           3900,    11,   484,   423,  1813,   257, 25627,  7955,   284,   257,
           4283,    13,  2102,    11,   262,  2176,  4283,   318,   407,  4750,
            287,   262, 10638,    13,   383,  2708,   635, 15802,  1194,  6509,
           5115,   257,  3482,  3331,    11,   475,   857,   407,  2148,  2252,
           3307,   393, 11986,   543,  3331,   318,   852,  7728,   546,    13,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50

In [58]:
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

dataloader = DataLoader(dataset, batch_size=8,shuffle=True)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

dataloader

In [59]:
from tqdm import tqdm
import gc

In [69]:
torch.cuda.empty_cache()
gc.collect()

0

In [70]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inp, lab in tqdm(dataloader):
        inputs = inp.to(device)
        labels = lab.to(device)
        # Forward pass
        outputs = model(inputs['input_ids'], labels=labels['input_ids'])
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')


100%|████████████████████████████████████████████████████████████████████████████████| 691/691 [01:46<00:00,  6.48it/s]


Epoch 1/100, Loss: 2.266766201914996


100%|████████████████████████████████████████████████████████████████████████████████| 691/691 [01:44<00:00,  6.62it/s]


Epoch 2/100, Loss: 1.7388358609548697


100%|████████████████████████████████████████████████████████████████████████████████| 691/691 [01:45<00:00,  6.57it/s]


...

Epoch 52/100, Loss: 0.0997991086066028


100%|████████████████████████████████████████████████████████████████████████████████| 691/691 [01:43<00:00,  6.68it/s]


Epoch 53/100, Loss: 0.09702168261112076


 33%|██████████████████████████▏                                                     | 226/691 [00:33<01:09,  6.66it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model, 'example_distiledGPT.pth')